In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from tqdm import tqdm
#scraping imports:
from imutils import paths
import argparse
import requests
import cv2
import os
import ctypes
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
import glob
import time

In [22]:
DATA = r"./data/top_anime_list/UsableAnimeList.xlsx"
df = pd.read_excel(DATA, sheet_name="usable")
anime_list = df["usable_title"].tolist()

In [23]:
class scrapeAnimeImageURLs:
    def __init__(self, anime_list):
        self.downloads_folder = r"C:\Users\kaush\Downloads"
        self.destination_folder = r"./data/anime_img_urls"
        for anime in anime_list:
            self.get_image_URLS(anime)
            self.move_download_file(anime)
    
    def get_image_URLS(self, anime_name):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless=new')
        self.driver = webdriver.Chrome(options=self.options)
        self.driver.get("https://www.google.com/search?tbm=isch&q=" + anime_name)
        self.driver.execute_script("window.scrollBy(0,10000)")
        time.sleep(0.5)
        self.driver.execute_script("window.scrollBy(0,20000)") # adding more of these will load more images
        time.sleep(0.5)
        self.driver.execute_script("window.scrollBy(0,30000)") # putting it in a loop somehow reduces the number of images and scrolls less
        time.sleep(0.5)
        self.driver.execute_script("window.scrollBy(0,40000)")
        time.sleep(0.5)
        self.driver.execute_script("window.scrollBy(0,50000)")
        time.sleep(0.5)
        self.driver.execute_script("window.scrollBy(0,60000)")
        time.sleep(0.5)
        self.driver.execute_script("window.scrollBy(0,70000)") # we'll get roughly 800 imags after all the scrolling
        time.sleep(0.5)
        self.driver.execute_script("window.scrollBy(0,80000)") # so its roughly 100 images per 10000 y-axis scroll
        self.driver.execute_script(r"""
            function saveImgTagToCSV(jsonArray) {
            const csvData = jsonArray.map((element) => {
                let imgTag = '';

                if (element.outerHTML) {
                imgTag = `"${element.outerHTML.replace(/"/g, '""')}"`;
                }

                return imgTag;
            }).join('\n');

            return csvData;
            }
            //then run this:
            const imgs_rg = Array.from(document.querySelectorAll('.rg_i'));
            const imgs_grn = Array.from(document.querySelectorAll('.GRN7Gc'));
            const imgs_ll = Array.from(document.querySelectorAll('.ll3EIMe'));
            const imgs = imgs_rg.concat(imgs_grn, imgs_ll);
            img_data = saveImgTagToCSV(imgs);
            window.open('data:text/csv;charset=utf-8,' + escape(img_data))""")
        
    def move_download_file(self, anime_name):
        time.sleep(1)
        file_list = glob.glob(os.path.sep.join([self.downloads_folder, "/*"]))
        latest_file = max(file_list, key=lambda x: os.stat(x).st_mtime)
        self.created_file = self.destination_folder+"/"+anime_name+".csv"
        os.replace(latest_file, self.created_file)
        
    def clean_urls(self):
        pass
    
        

In [24]:
scraper = scrapeAnimeImageURLs(["cowboy bepop", "naruto"])

In [20]:
# options = webdriver.ChromeOptions()
# #options.add_argument('headless')
# options.add_argument('window-size=1200x600')
# options.add_argument('--headless=new')
# driver = webdriver.Chrome(options=options)
# driver.get("https://www.google.com/search?tbm=isch&q=" + "cowboy bebop")
# driver.execute_script("window.scrollBy(0,10000)")
# time.sleep(0.5)
# driver.execute_script("window.scrollBy(0,20000)") #adding more of these will load more images
# time.sleep(0.5)
# driver.execute_script("window.scrollBy(0,30000)") #putting it in a loop somehow reduces the number of images and scrolls less
# time.sleep(0.5)
# driver.execute_script("window.scrollBy(0,40000)")
# time.sleep(0.5)
# driver.execute_script("window.scrollBy(0,50000)")
# time.sleep(0.5)
# driver.execute_script("window.scrollBy(0,60000)")
# time.sleep(0.5)
# driver.execute_script("window.scrollBy(0,70000)")
# time.sleep(0.5)
# driver.execute_script("window.scrollBy(0,80000)")
# driver.execute_script(r"""
# function saveImgTagToCSV(jsonArray) {
#   const csvData = jsonArray.map((element) => {
#     let imgTag = '';

#     if (element.outerHTML) {
#       imgTag = `"${element.outerHTML.replace(/"/g, '""')}"`;
#     }

#     return imgTag;
#   }).join('\n');

#   return csvData;
# }
#   //then run this:
#   const imgs_rg = Array.from(document.querySelectorAll('.rg_i'));
#   const imgs_grn = Array.from(document.querySelectorAll('.GRN7Gc'));
#   const imgs_ll = Array.from(document.querySelectorAll('.ll3EIMe'));
#   const imgs = imgs_rg.concat(imgs_grn, imgs_ll);
#   img_data = saveImgTagToCSV(imgs);
#   window.open('data:text/csv;charset=utf-8,' + escape(img_data))""")

    Image classes:
        rg_i Q4LuWd
        GRN7Gc
        ll3IMe.CBvy7c
    Somehow we always have 0 LL images, but that's a worry for later....I'll keep it there just in case it works :D

    
    Current workflow:
    Run the driver line above
    Run the following in the console:
    thanks ChatGPT for helping when I was stuck with JavaScript